In [2]:
import json
import os
from io import BytesIO

import folium
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
import rasterio
import requests
from owslib.wfs import WebFeatureService
from pyproj import CRS
from rasterio.transform import from_bounds
from rasterstats import zonal_stats
from shapely.geometry import Point, box, shape

In [ ]:
def list_wfs_layers(wfs_url):
    wfs = WebFeatureService(url=wfs_url, version='2.0.0')
    print("Service title:", wfs.identification.title)
    for name, md in wfs.contents.items():
        print(f"- {name} :: {getattr(md, 'title', '')}")

In [4]:
list_wfs_layers("https://geo.api.vlaanderen.be/Gebouwenregister/wfs")

Service title: WFS Gebouwenregister
- Gebouwenregister:Gebouw :: Gebouw
- Gebouwenregister:Gebouweenheid :: Gebouweenheid


In [10]:
OUT_DIR = "data_leuven"
TARGET_EPSG = 31370
WGS84 = 4326

GRB_WFS_URL = "https://geo.api.vlaanderen.be/Gebouwenregister/wfs"
GRB_WFS_LAYER = "Gebouwenregister:Gebouw"

LARGE_ROOF_MIN_M2 = 500
FILL_FACTOR = 0.6
KWH_PER_M2 = 200.0
CO2_KG_PER_KWH = 0.23

In [11]:
def get_leuven_boundary(place="Leuven, Belgium", epsg=TARGET_EPSG):
    import osmnx as ox
    g = ox.geocode_to_gdf(place).to_crs(epsg)
    return g.dissolve().reset_index(drop=True)

def fetch_buildings_wfs(leuven_gdf, url, layer, epsg=TARGET_EPSG):
    wfs = WebFeatureService(url=url, version="1.1.0")
    minx, miny, maxx, maxy = leuven_gdf.total_bounds
    resp = wfs.getfeature(typename=layer,
                          bbox=(minx, miny, maxx, maxy, f"EPSG:{epsg}"),
                          outputFormat="json")
    gdf = gpd.read_file(BytesIO(resp.read())).to_crs(epsg)
    return gdf

def clean_columns(g):
    g = g.copy()
    ren = {}
    if "id" in g.columns: ren["id"] = "src_id"
    if "Id" in g.columns: ren["Id"] = "wfs_id"
    if "ObjectId" in g.columns: ren["ObjectId"] = "object_id"
    g = g.rename(columns=ren)
    if "VersieId" in g.columns:
        g["VersieId"] = pd.to_datetime(g["VersieId"], utc=True, errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")
    g = g[g.geometry.notnull() & g.geometry.is_valid]
    return g

def add_area_centroid(g):
    g = g.copy()
    g["area_m2"] = g.geometry.area
    cen = g.geometry.centroid
    g["centroid_x"] = cen.x
    g["centroid_y"] = cen.y
    return g

def make_geojson_safe(g):
    g = g.copy().drop(columns=["VersieId"], errors="ignore")
    for c in g.columns:
        if c != "geometry" and pd.api.types.is_datetime64_any_dtype(g[c]):
            g[c] = pd.to_datetime(g[c], errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")
    return g

def save_layers(gdf, name, out_dir=OUT_DIR):
    os.makedirs(out_dir, exist_ok=True)
    gdf.to_file(os.path.join(out_dir, f"{name}.gpkg"), driver="GPKG", engine="fiona", index=False)
    gdf.to_file(os.path.join(out_dir, f"{name}.geojson"), driver="GeoJSON", engine="fiona")
    gdf.drop(columns="geometry").to_csv(os.path.join(out_dir, f"{name}.csv"), index=False)

In [12]:
leuven = get_leuven_boundary()
buildings_raw = fetch_buildings_wfs(leuven, GRB_WFS_URL, GRB_WFS_LAYER)
buildings = add_area_centroid(clean_columns(buildings_raw))

candidates = buildings[buildings["area_m2"] >= LARGE_ROOF_MIN_M2].copy()

candidates["panel_m2"] = candidates["area_m2"] * FILL_FACTOR
candidates["kwh_year"] = candidates["panel_m2"] * KWH_PER_M2
candidates["co2_tons"] = candidates["kwh_year"] * CO2_KG_PER_KWH / 1000.0
candidates["rank"] = candidates["co2_tons"].rank(ascending=False, method="dense").astype(int)
top200 = candidates.sort_values("co2_tons", ascending=False).head(200).copy()

In [13]:
candidates

,src_id,wfs_id,object_id,VersieId,GeometrieMethode,GebouwStatus,geometry,area_m2,centroid_x,centroid_y,panel_m2,kwh_year,co2_tons,rank
180,Gebouw.16580848,https://data.vlaanderen.be/id/gebouw/16580848,16580848,2023-11-03 20:01:35,IngemetenGRB,Gerealiseerd,"POLYGON ((170566.247 181683.711, 170563.079 18...",11342.750736,170512.024489,181694.909534,6805.650442,1.361130e+06,313.059920,3
256,Gebouw.31592581,https://data.vlaanderen.be/id/gebouw/31592581,31592581,2025-06-13 08:43:31,IngemetenGRB,Gerealiseerd,"POLYGON ((175426.517 174562.727, 175425.676 17...",502.544103,175408.502967,174566.412367,301.526462,6.030529e+04,13.870217,230
267,Gebouw.14066971,https://data.vlaanderen.be/id/gebouw/14066971,14066971,2023-11-02 17:01:30,IngemetenGRB,Gehistoreerd,"POLYGON ((172858.039 172983.715, 172843.314 17...",1093.768753,172834.643165,172984.155251,656.261252,1.312523e+05,30.188018,101
280,Gebouw.20591951,https://data.vlaanderen.be/id/gebouw/20591951,20591951,2025-06-17 05:23:05,IngemetenGRB,Gerealiseerd,"POLYGON ((171605.231 174511.563, 171605.597 17...",64277.724339,171407.692197,174258.310655,38566.634603,7.713327e+06,1774.065192,1
389,Gebouw.14174954,https://data.vlaanderen.be/id/gebouw/14174954,14174954,2023-11-02 16:10:50,IngemetenGRB,Gerealiseerd,"POLYGON ((171735.803 172812.609, 171734.307 17...",1396.887892,171740.653156,172791.875141,838.132735,1.676265e+05,38.554106,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9859,Gebouw.16830557,https://data.vlaanderen.be/id/gebouw/16830557,16830557,2023-11-03 10:16:40,IngemetenGRB,Gerealiseerd,"POLYGON ((170328.574 181572.873, 170324.817 18...",853.301802,170302.265104,181571.059617,511.981081,1.023962e+05,23.551130,132
9909,Gebouw.19741568,https://data.vlaanderen.be/id/gebouw/19741568,19741568,2023-11-03 04:57:48,IngemetenGRB,Gerealiseerd,"POLYGON ((174609.033 175166.101, 174602.758 17...",664.793458,174608.985128,175151.660670,398.876075,7.977522e+04,18.348299,176
9937,Gebouw.12256261,https://data.vlaanderen.be/id/gebouw/12256261,12256261,2023-11-03 20:18:21,IngemetenGRB,Gerealiseerd,"POLYGON ((177260.381 168797.607, 177263.63 168...",771.918343,177248.426247,168785.735717,463.151006,9.263020e+04,21.304946,145
9972,Gebouw.14571733,https://data.vlaanderen.be/id/gebouw/14571733,14571733,2023-11-02 15:44:34,IngemetenGRB,Gerealiseerd,"POLYGON ((174761.029 171105.835, 174763.596 17...",4160.738713,174735.386578,171130.319673,2496.443228,4.992886e+05,114.836388,16


In [14]:
top200

,src_id,wfs_id,object_id,VersieId,GeometrieMethode,GebouwStatus,geometry,area_m2,centroid_x,centroid_y,panel_m2,kwh_year,co2_tons,rank
280,Gebouw.20591951,https://data.vlaanderen.be/id/gebouw/20591951,20591951,2025-06-17 05:23:05,IngemetenGRB,Gerealiseerd,"POLYGON ((171605.231 174511.563, 171605.597 17...",64277.724339,171407.692197,174258.310655,38566.634603,7.713327e+06,1774.065192,1
3064,Gebouw.19740151,https://data.vlaanderen.be/id/gebouw/19740151,19740151,2024-04-15 17:06:30,IngemetenGRB,Gerealiseerd,"POLYGON ((173095.268 179309.44, 173100.055 179...",19750.471188,173124.163178,179225.929226,11850.282713,2.370057e+06,545.113005,2
180,Gebouw.16580848,https://data.vlaanderen.be/id/gebouw/16580848,16580848,2023-11-03 20:01:35,IngemetenGRB,Gerealiseerd,"POLYGON ((170566.247 181683.711, 170563.079 18...",11342.750736,170512.024489,181694.909534,6805.650442,1.361130e+06,313.059920,3
5282,Gebouw.19697068,https://data.vlaanderen.be/id/gebouw/19697068,19697068,2023-11-03 04:05:57,IngemetenGRB,Gerealiseerd,"POLYGON ((171269.387 175384.693, 171249.894 17...",8345.700857,171291.775636,175336.389928,5007.420514,1.001484e+06,230.341344,4
1101,Gebouw.14572288,https://data.vlaanderen.be/id/gebouw/14572288,14572288,2025-09-17 09:18:33,IngemetenGRB,Gerealiseerd,"POLYGON ((172001.22 174728.593, 172001.04 1747...",8147.909898,171981.865270,174663.270368,4888.745939,9.777492e+05,224.882313,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8757,Gebouw.19744914,https://data.vlaanderen.be/id/gebouw/19744914,19744914,2023-11-03 05:05:03,IngemetenGRB,Gerealiseerd,"POLYGON ((174644.91 170959.588, 174645.703 170...",607.160333,174630.397901,170950.460582,364.296200,7.285924e+04,16.757625,196
9815,Gebouw.19743191,https://data.vlaanderen.be/id/gebouw/19743191,19743191,2023-11-03 05:04:36,IngemetenGRB,Gerealiseerd,"POLYGON ((174200.748 174953.691, 174193.167 17...",603.591546,174178.150435,174945.845894,362.154927,7.243099e+04,16.659127,197
7813,Gebouw.12156204,https://data.vlaanderen.be/id/gebouw/12156204,12156204,2023-11-03 19:39:05,IngemetenGRB,Gehistoreerd,"POLYGON ((177543.384 169608.374, 177531.358 16...",601.610034,177551.626868,169595.822894,360.966020,7.219320e+04,16.604437,198
7334,Gebouw.31618286,https://data.vlaanderen.be/id/gebouw/31618286,31618286,2025-10-26 23:15:38,IngemetenGRB,Gerealiseerd,"POLYGON ((172498.599 174631.655, 172522.938 17...",593.249484,172508.084946,174618.564430,355.949691,7.118994e+04,16.373686,199


In [15]:
save_layers(buildings, "leuven_buildings")
save_layers(candidates, "leuven_large_roofs")
save_layers(top200, "leuven_top200_roofs")

In [16]:
candidates_vis = make_geojson_safe(candidates).to_crs(4326)
minx, miny, maxx, maxy = candidates_vis.total_bounds
m = folium.Map(tiles="OpenStreetMap")
m.fit_bounds([[miny, minx], [maxy, maxx]])

folium.GeoJson(
    data=candidates_vis.__geo_interface__,
    name="Large roofs",
    style_function=lambda f: {"color": "red", "weight": 2, "fillOpacity": 0.35}
).add_to(m)

if len(top200):
    t200 = make_geojson_safe(top200).to_crs(4326)
    folium.GeoJson(
        data=t200.__geo_interface__,
        name="Top 200 (by CO₂)",
        style_function=lambda f: {"color": "blue", "weight": 2, "fillOpacity": 0.35}
    ).add_to(m)

folium.LayerControl().add_to(m)
m.save(os.path.join(OUT_DIR, "leuven_roofs.html"))

In [17]:
m